Make sure that libraries are already installed.

In [13]:
import platform
import os, sys
import numpy as np
from osgeo import gdal
gdal.UseExceptions()     

print("Python Version",platform.python_version())
print("GDAL Version", gdal.__version__)

Python Version 3.6.7
GDAL Version 3.0.0


As long as Sentinel-2 data is stored in [JPEG2000 format](https://github.com/uclouvain/openjpeg), we need the "driver" that understands this image compression format which is known as a .jp2 file. So, we have to check it's installation as well.

In [11]:
!gdalinfo --formats | grep JPEG2000

  JPEG2000 -raster,vector- (rwv): JPEG-2000 part 1 (ISO/IEC 15444-1), based on Jasper library


Get testing data

Download from copernicus the following files:

T18NZN_20181217T151659_TCI_10m.jp2

T18NZN_20181222T151701_TCI_10m.jp2

T18NZN_20190106T151659_TCI_10m.jp2

T18NZN_20190111T151701_TCI_10m.jp2

T18NZN_20190126T151709_TCI_10m.jp2

In [6]:
!gdalinfo ../tiler/T18NZN_20181217T151659_TCI_10m.jp2

Driver: JPEG2000/JPEG-2000 part 1 (ISO/IEC 15444-1), based on Jasper library
Files: ../tiler/T18NZN_20181217T151659_TCI_10m.jp2
Size is 10980, 10980
Coordinate System is:
PROJCS["WGS 84 / UTM zone 18N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",-75],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    AXIS["Easting",EAST],
    AXIS["Northing",NORTH],
    AUTHORITY["EPSG","32618"]]
Origin = (799980.000000000000000,800040.000000000000000)
Pixe

In [2]:
dset = gdal.Open('../tiler/T18NZN_20181217T151659_TCI_10m.jp2')
##print(dset)
width = dset.RasterXSize
height = dset.RasterYSize

print(width, 'x', height)

10980 x 10980


<bound method MajorObject.GetMetadata of <osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f7f9d340f00> >>

In order to visualize on the notebook the images, I've built a function to load pixels.

In [16]:
import cv2
import glob
import rasterio
from rasterio import plot
import matplotlib.pyplot as plt
%matplotlib inline
#imagePath = '../data/2019-01/S2B_MSIL1C_20190129T152639_N0207_R025_T18PYQ_20190129T201832.SAFE/GRANULE/L1C_T18PYQ_A009918_20190129T152639/IMG_DATA/'

In [15]:
def load_data_bands(path,bands='bgrn'):
    bmd = {'bgrn':['02','03','04','08'],
           'rgb':['04','03','02'],
           'all':['01','02','03','04','05','06','07','08','09','10','11','12']} #dictionary bands map
    arrs = []
    meta = None
    try:
        jp2s=bmd[bands]
        for jp2 in jp2s:
            with rasterio.open(glob.glob(path+'*B'+jp2+'.jp2')[0]) as f:
                print(f)
                arrs.append(f.read(1))
            if jp2 == '02':
                meta = f.meta
        if bands in {'bgrn','rgb'}: 
            data = np.array(arrs,dtype=arrs[0].dtype)
        else: 
            data = arrs
        return data, meta
    except:
        print('bands map not defined')
        return None, None